## bitMEX trading test

目标：跑通bitMEX交易

策略：专供测试交易系统的随机策略RandomTargetPostionStrategy：每隔30s发出一个随机的SignalEvent。

In [13]:
import logging
import time
import queue

In [ ]:
class NaivePortfolio(object):
    """简单CTA策略的组合管理器
    
    将各个CTA子策略的signal汇总，计算汇总的target_position， (目前是单一identifier)
    当实际仓位与理论仓位不一致时发出OrderEvent，并处理FillEvent。
    """
    
    def __init__(self, event_queue, is_test=True):
        
        self.is_test = True   # 是否测试
        self.logger = logging.getLogger()   # 日志
        
        self.event_queue = event_queue
        
        self.symbol = 'XBTUSD'
        self.lots = 20
        
        # 目标仓位。
        # 当实际仓位与目标仓位不一致时，会触发下单逻辑。
        self.target_position = 0
        
        # 实际仓位。
        # 1. 初始化时会主动查询 self.query_positon()  
        # 2. 收到成交回报时会更新。  -> 如果程序开着时手动下单，那么程序会收到成交回报，把仓位调回到目标仓位。
        self.actual_position = None         
        
    def onSignalEvent(self, signalEvent):
        """
        signalEvent Handler
        
        signalEvent格式:
        {
            identifier: 'Turtle_XBTUSD_1m_9999',
            symbol: 'XBTUSD',
            direction: -1,
        }
        """
        self.logger.info('signalEvent: %s' % signalEvent)
        self._signal_event_handler(signalEvent)   # 处理signal_event
        
    def _signal_event_handler(signalEvent):
        if not signalEvent.get('symbol') == self.symbol:
            self.logger.error('signalEvent.symbol != bitmexTrader.symbol')
            return 
        if signalEvent.get('direction') in (0, -1, 1):
            self.logger.error('invalid signalEvent.direction: %s')
            return
        self.target_position = signalEvent['direction'] * self.lots   # 设定 target_position
        self._trade_to_target()   # 交易
        
    def _trade_to_target(self):
        position_diff = self.target_position - self.actual_position
        if position_diff > 0:
            direction = 'buy'
            limit_price = self.ask1  # 对价下单
        elif position_diff < 0:
            direction = 'sell'
            limit_price = self.bid1   # 对价下单
        else:
            self.logger.info('target==actual, no need to trade')
            return 0
        symbol=self.symbol
        volume=abs(position_diff)
        order_event = orderEvent(symbol, direction, volume, limit_price)   # 构造Order
        
        # TODO: 如果有未执行的委托单怎么办？
        self.event_queue.put(order_event)

重新理清思路：

portfolio 只负责维护 target_position. 发出 target_position_event -> 提示target_position可能有变化。
它的主要功能后续应该是汇总各个子策略的仓位。

executor 负责维护actual_position. 处理发单、撤单、查询持仓。

第一阶段任务，先以最简单的方式，跑通bitMEX实盘交易：
- portofolio线程: 随机生成 target_position 放入队列
- executor线程: 循环从队列中取事件，交易

## Demo of placing an order

**BitMEX 不支持通过 WebSocket 提交或取消委托，这些操作只能通过 HTTP 进行。**

我们的服务器支持保持 HTTP 连接和缓存的 SSL 会话。 如果你保持一个有效连接，你会得到与 websocket 类似的延迟，而无须使用 websocket 进行沟通。

我们保持活动状态的超时时间为90秒。

## bitmexREST

## BitMEX WebSocket

copy form https://github.com/BitMEX/api-connectors/ and https://github.com/BitMEX/sample-market-maker

## bitmex TargetPosition-Based OMS

TODO: seperate bitmex gateway and TargetPosition-Based OMS

- bitmexWS
    + 行情（无需验证）
    + 交易。实时持仓情况、下单回报、成交回报
- bitmexREST
    + 下单
    + 主动持仓查询。仅用于定时校验。
    + 查历史行情（无需验证）。仅用于策略程序启动时回看。
    

OMS has three key member：

- `bm_ws_market`  subclass of `bitmexWS`. 【行情】收行情，包括 trades, orderbook, (funding, markprice etc.)
- `bm_ws_trading` subclass of `bitmexWS`. 【交易】监听下单回报、成交回报、持仓变动 （保证金）
- `bm_rest` instance of `bitmexREST`. 下单函数 `place_order()`，查询函数`get_positions()`


`bm_ws_market` subscribe `trade:XBTUSD` and override `onData()`, which do 2 things: 
1. upate the 4 members  (members: last_price, bar, prev_bar, order_book etc.)
2. generate MARKET_EVENT when needed(eg. on_bar_close)

`bm_ws_trading` subscribe `order, position, execution` and override `onData()`, which 
1. update 2 members (unfilled_order, actual_position)
2. generate ORDER_EVENT, FILL_EVENT

`bm_rest` is used to `place_order()`, do not generate or listen to or handle any event. Another function is query and check positions from WS.

and one important function `trade_to_target()`, which make order placing and/or order canceling decision.

> 要不这样：     
> 把bm_ws_market和bm_rest的一个实例打包成 `bitmexDataHandler`    
> 把 bm_ws_trading和bm_rest的另一个实例打包成  `bitmexTrader`

### bitmexWS - Market

In [1]:
from bitmexWS import bitmexWS
from bitmexREST import bitmexREST
from utils import generate_logger

#from qsEvent import MarketEvent
#from qsObject import Bar
#from utils import calculate_ts

class MarketEvent(object):
    def __init__(self, data):
        self.etype='MARKET'
        self.data = data
       
    
class Bar(object):
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __repr__(self):
        return self.__dict__.__repr__()
    
    
def calculate_ts(timestamp, bar_type='1m'):
    """bitmex-timestamp to ts. eg 2018-09-29T06:00:17.271Z -> 20180929060017"""
    return timestamp[:16].replace('T', ' ')  # TODO: now only allow '1m'

class bitmexWSMarket(bitmexWS):
    """bitmexWS subscribing market data (single symbol)"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.logger = generate_logger('bitmexWS_Market')
    
    def addEventQueue(self, q):
        self.eventQueue = q

    def subscribe(self, symbol='XBTUSD', bar_type='1m', subscribe_tick=True):
        self.symbol = symbol
        self.bar_type = bar_type
        self.subscribe_tick = subscribe_tick
        self.subscribe_topic('trade:%s' % symbol)  # TODO: subscribe more topic, such as order book
        self._got_partial = False
        self.last_price = None
        
    def wait_for_lastprice(self):
        while self.last_price is None:
            self.logger.debug('waiting for lastprice ...')
            time.sleep(1)
        
    def onData(self, msg):
        if self._got_partial:
            self._handle_data(msg['data'])   # msg['data'] is a list of dict
        else:
            if msg['action'] == 'partial':
                self._got_partial = True
                self._handle_init_data(msg['data'])
                self.logger.debug('Got partial.')
            elif msg['action'] == 'insert':
                self.logger.debug('drop data before partial')
    
    def _handle_init_data(self, data):
        tick_d = data[-1]
        ts = calculate_ts(tick_d['timestamp'], self.bar_type)
        tick_price = tick_d['price']
        self.last_price = tick_price
        init_bar = Bar(open=tick_price, high=tick_price, low=tick_price, ts=ts)
        self.current_bar = init_bar
        self.logger.debug('init_bar: %s' % init_bar)
    
    def _handle_data(self, data):
        # data is list of dict: [{}, {}]
        # eg 'data': [{'symbol': 'XBTUSD', 'tickDirection': 'PlusTick', 'timestamp': '2018-09-29T06:00:17.271Z', 'price': 6484.5, 'trdMatchID': '87bd9b19-6747-c804-3eae-7a84fc7abcf8', 'foreignNotional': 30, 'grossValue': 462630, 'homeNotional': 0.0046263, 'side': 'Buy', 'size': 30}]
        for tick_d in data:
            self._on_tick(tick_d)
    
    def _on_tick(self, tick_d):
        tick_price = tick_d['price']
        
        # 更新last_price
        self.last_tick_price = self.last_price   # move:self.last_price  -> self.last_tick_price
        self.last_price = tick_price   # **current**_tick_price
        
        # bar-generator
        ts = calculate_ts(tick_d['timestamp'], self.bar_type)  # 'timestamp': '2018-09-29T06:00:17.271Z'
        if ts > self.current_bar.ts:
            # bar_close event
            self.current_bar.close = self.last_tick_price            
            self.prev_bar = self.current_bar
            self.current_bar = Bar(open=tick_price, high=tick_price, low=tick_price, ts=ts)
            
            bar_close_event = MarketEvent(data={'type': 'BAR_CLOSE'})
            self.eventQueue.put(bar_close_event)
            self.logger.debug('bar_close event. prev_bar is %s' % self.prev_bar)
            
            
            # bar_open event
            bar_open_event = MarketEvent(data={'type': 'BAR_OPEN'})
            self.eventQueue.put(bar_open_event)
            self.logger.debug('bar_open event. current_bar is %s' % self.current_bar)
        else:
            self.current_bar.high = max(self.current_bar.high, tick_price)
            self.current_bar.low = min(self.current_bar.low, tick_price)
        # tick event    
        if self.subscribe_tick:
            tick_event = MarketEvent(data={'type':'TICK'})
            self.eventQueue.put(tick_event)
            self.logger.debug('tick event: last_price is %s' % tick_d['price'])
        

## bitmexWSTrading

In [2]:
# summarize TODOS:
# class bitmexWSTrading   # Done
#   subscribe()
#   wait_for_initial_status()
#   actual_position
#   unfilled_qty
# class bitmexWSMarket   # delay. use last_price + slippage
#   bid1, ask1
# qsObject.py
#   class Order

In [3]:
from bitmexWS import bitmexWS
from bitmexREST import bitmexREST
from utils import generate_logger


class bitmexWSTrading(bitmexWS):
    """bitmex Websockt subscribing topics related to live trading"""
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.logger = generate_logger('bitmexWS_Trading')

    def subscribe(self, symbols=('XBTUSD', 'ETHUSD')):
        self.symbols = symbols
        
        self.actual_position = {s:0 for s in symbols}                    # {symbol: pos}   str:int
        self.unfilled_qty = {s:{'Buy': 0, 'Sell': 0} for s in symbols}    # {symbol: {'Buy': qty, 'Sell': qty}}
        
        self._got_position_partial = False
        
        s = ','.join(self.symbols)
        self.subscribe_topic('order:%s' % s)
        self.subscribe_topic('position:%s' % s)
        self.subscribe_topic('execution:%s' % s)
        
    def onData(self, msg):
        if msg.get('table') == 'position':
            self._on_position_msg(msg)
        elif msg.get('table') == 'order':
            self.__on_order_msg(msg)
        elif msg.get('table') == 'execution':
            self.__on_execution_msg(msg)
    
    def _on_position_msg(self, msg):
        self.logger.info('Got position msg:')
        #print('========================position==================\n' + msg.__str__())
        
        #global g_msg
        #g_msg = copy.deepcopy(msg)  ###### 将第一个遇到的msg存入全局变量，调试用
        
        if self._got_position_partial and msg['action'] == 'update':

            if msg['data']:
                
                for d in msg['data']:
                    symbol = d.get('symbol')
                    if symbol not in self.symbols:
                        self.logger.warning('Got position subscription of symbol: %s, not in self.symbols' % symbol)
                        continue
                    currentQty = d.get('currentQty')
                    openOrderBuyQty = d.get('openOrderBuyQty', None)
                    openOrderSellQty = d.get('openOrderSellQty', None)
                    
                    old_pos = self.actual_position.get(symbol)
                    old_buy_qty = self.unfilled_qty.get(symbol, {}).get('Buy')
                    old_sell_qty = self.unfilled_qty.get(symbol, {}).get('Sell')
                    
                    if old_pos != currentQty:
                        self.actual_position[symbol] = currentQty   
                        self.logger.info('█████ Position update (actual_position) █████ %s: %s -> %s' % (symbol, old_pos, currentQty))
                    if openOrderBuyQty is not None:
                        self.unfilled_qty[symbol]['Buy'] = openOrderBuyQty
                        self.logger.info('███ Position update (unfilled_qty, Buy) ███ %s: %s -> %s' % (symbol, old_buy_qty, openOrderBuyQty))
                    if openOrderSellQty is not None:
                        self.unfilled_qty[symbol]['Sell'] = openOrderSellQty
                        self.logger.info('███ Position update (unfilled_qty, Sell) ███ %s: %s -> %s' % (symbol, old_sell_qty, openOrderSellQty))
            else:
                self.logger.debug('#### Position update #### is []')
            
        elif msg['action'] == 'partial':
            self._got_position_partial = True
            
            if msg['data']:
                for d in msg['data']:
                    symbol = d.get('symbol')
                    if symbol not in self.symbols:
                        self.logger.warning('Got position subscription of symbol: %s, not in self.symbols' % symbol)
                        continue
                    currentQty = d.get('currentQty')
                    openOrderBuyQty = d.get('openOrderBuyQty', 0)                    
                    openOrderSellQty = d.get('openOrderSellQty', 0)
                    
                    self.actual_position[symbol] = currentQty
                    self.unfilled_qty[symbol]['Buy'] = openOrderBuyQty
                    self.unfilled_qty[symbol]['Sell'] = openOrderSellQty
                    
                    txt = '%s  pos: %s, buy: %s, sell: %s' % (symbol, currentQty, openOrderBuyQty, openOrderSellQty)
                    self.logger.debug('██████████ Position partial ██████████ %s' % txt)
            else:
                self.logger.debug('██████████  Position partial ██████████  is []')
            
        
        
    def __on_order_msg(self, msg):
        self.logger.info('Got order msg')
        #print('========================order==================\n' + msg.__str__())
        
    def __on_execution_msg(self, msg):
        self.logger.info('Got execution msg')
        #print('========================execution==================\n' + msg.__str__())
            
        
    def wait_for_initial_status(self):
        if self._got_position_partial:
            return
        else:
            time.sleep(1)

### bitmex REST

### OMS class

In [4]:
from utils import generate_logger


class bitmexTargetPositionOMS(object):
    """bitmex TargetPosition-Based Order Management System"""
    
    def __init__(self, bm_ws_market, eventQueue, apiKey, apiSecret, symbols):
        
        # 日志
        self.logger = generate_logger('OMS')
        
        # 绑定事件队列， 队列中只有 TARGET_POSITION_EVENT，另开一个线程来push目标仓位
        self.eventQueue = eventQueue
        
        # 目标仓位 {symbol: pos}
        self.target_position = {}
        
        # 标的
        self.symbols = symbols
        
        # websocket-market
        self.bm_ws_market = bm_ws_market   # 外部的，因为DataHandler同时也在用它 or 它就是DataHandler
        
        # websocket-trading
        self.bm_ws_trading = bitmexWSTrading(apiKey, apiSecret)
        self.bm_ws_trading.connect()
        self.bm_ws_trading.subscribe(self.symbols)
        self.bm_ws_trading.wait_for_initial_status()  # 等待的初始信息
        self.actual_position = self.bm_ws_trading.actual_position  # 由websocket接收的信息计算出的实际仓位 `position`
        self.unfilled_qty = self.bm_ws_trading.unfilled_qty  # 由websocket接收的信息计算出的未成交委托  `order`
        
        # rest
        self.bm_rest = bitmexREST(apiKey, apiSecret)
        
    def exit(self):
        self.bm_ws_trading.exit()
        self.bm_ws_market.exit()
        
    def set_target_position(self, symbol, position):
        self.target_position[symbol] = position      
        
    def trade_to_target(self, symbol):
        if symbol not in self.symbols:
            self.logger.warning('Calling `trade_to_target` but arg `symbol` is not in self.symbols\n' + 
                                'symbol=%s\n' + 
                                'self.symbols=%s' % (symbol, self.symbols))
            
        target_pos = self.target_position.get(symbol)  # int
        actual_pos = self.actual_position.get(symbol, 0)  # int
        
        if target_pos is None:
            self.logger.warning('Calling `trade_to_target()` but arg `symbol` is not in self.target_position\n' + 
                                'symbol=%\n' + 
                                'self.target_position=%s' % (symbol, self.target_position))
        
        # 这里采用比较暴力的办法：直接cancel_all_orders, 再挂目标仓位与实际仓位差值的单子
        # 有优化的空间，eg. bitmex支持改单；
        if target_pos == actual_pos:
            unfilled_qty = self.unfilled_qty[symbol]  # {'Buy': 1, 'Sell': 1}
            total_unfilled_qty = sum([abs(x) for x in unfilled_qty.values()])
            if total_unfilled_qty == 0:
                self.logger.info('target_pos == actual_pos && unfilled_qty is 0, nothing to do')
            else:
                self.bm_rest.cancel_all_order(symbol)
        else:
            self.bm_rest.cancel_all_orders(symbol)  # 先全撤掉
            # 构造order
            pos_diff = target_pos - actual_pos
            side = 'Buy' if pos_diff > 0 else 'Sell'
            slippage = 0.5 * 5                  # 测试：5个滑点
            drc = 1 if side == 'Buy' else -1
            price = self.bm_ws_market.last_price + drc * slippage
            # 下单
            try:
                res = self.bm_rest.place_order(symbol=symbol, side=side, qty=abs(pos_diff), limit_price=price)
            except Exception as e:
                print('When placing order, an Error raise:\n %s' % e)
            else:                
                if res.ok:
                    self.logger.info('Successfully Place Order:\n%s' % res.json())
                else:
                    self.logger.info('Placeing Order Failed:\n%s' % res.json())
                
    def _check_actual_position_with_rest(self):
        """use REST api to query actual_position, check it with self.actual_position. Use a Thread to do this"""
        pass

In [5]:
## thread to push TARGET_POSITION_EVENT

import threading
import numpy as np
import queue
import time

# global
event_q = queue.Queue()  # 事件队列
symbol = 'XBTUSD'  # symbol

def generate_random_target_positon():
    """随机生成target_position的函数"""
    
    global event_q
    global symbol
    
    symbols = (symbol,)  #('XBTUSD', 'ETHUSD')
    n_symbols = len(symbols)
    
    while True:
        pos = np.random.normal(0, 5, n_symbols).astype(int)
        e = {
            'etype': 'TARGET_POSITION_EVENT',
            'data': {s:int(p) for s,p in zip(symbols, pos)}   # NOTE: np.int64 is not json serilizable
        }
        event_q.put(e)
        time.sleep(15)


# 开一个线程，专门生成 target_position event
target_position_td = threading.Thread(target=generate_random_target_positon, args=())
target_position_td.start()


############################ OMS ############################

# market
bm_ws_market = bitmexWSMarket()
bm_ws_market.addEventQueue(event_q)
bm_ws_market.connect()
bm_ws_market.subscribe(symbol)   # 目前只实现了单品种
bm_ws_market.wait_for_lastprice()  

# apiKey, apiSecret
import json


with open('accounts.json') as f:
        acc = json.load(f)

apiKey = acc[0]['apiKey']
apiSecret = acc[0]['apiSecret']


oms = bitmexTargetPositionOMS(bm_ws_market, event_q, apiKey, apiSecret, (symbol,))


# # 一次性测试 trade_to_target()
# oms.set_target_position('XBTUSD', 7)
# oms.trade_to_target('XBTUSD')



# 主程序

# oms.run()  # todo: encapsulate into oms.run()

while True:
    try:
        event = event_q.get_nowait()
    except queue.Empty:
        time.sleep(0.5)
    else:
        if isinstance(event, dict) and event['etype'] == 'TARGET_POSITION_EVENT':
            for sym, pos in event['data'].items():
                oms.set_target_position(sym, pos)  # 设定目标仓位
                print('▲▲▲▲▲ target_posion: %s ▲▲▲▲▲' % (oms.target_position))
            oms.trade_to_target(symbol)    # 交易
        else:
            print('Event: %s' % event.__repr__())

[2018-10-05 15:22:22 bitmexWS_Market INFO] ws thread start
[2018-10-05 15:22:24 bitmexWS_Market DEBUG] Calling ws.__on_open()
[2018-10-05 15:22:24 bitmexWS_Market INFO] Successful connected to BitMEX WebSocket API
[2018-10-05 15:22:24 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:22:24 bitmexWS_Market DEBUG] waiting for lastprice ...
[2018-10-05 15:22:24 bitmexWS_Market INFO] Subscribe to trade:XBTUSD
[2018-10-05 15:22:24 bitmexWS_Market DEBUG] init_bar: {'low': 6544, 'high': 6544, 'open': 6544, 'ts': '2018-10-05 07:22'}
[2018-10-05 15:22:24 bitmexWS_Market DEBUG] Got partial.
[2018-10-05 15:22:25 bitmexWS_Trading INFO] ws thread start
[2018-10-05 15:22:27 bitmexWS_Trading DEBUG] Calling ws.__on_open()
[2018-10-05 15:22:27 bitmexWS_Trading INFO] Successful connected to BitMEX WebSocket API
[2018-10-05 15:22:27 bitmexWS_Trading DEBUG] >>> send ping
[2018-10-05 15:22:28 bitmexWS_Trading INFO] Subscribe to order:XBTUSD
[2018-10-05 15:22:28 bitmexWS_Trading INFO] Subscribe to positio

▲▲▲▲▲ target_posion: {'XBTUSD': 3} ▲▲▲▲▲


[2018-10-05 15:22:29 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:22:30 bitmexREST INFO] Placing Order... post_dict is {'text': '[API][2018-10-05 15:22:30.645134] ', 'orderQty': 4, 'price': 6541.5, 'symbol': 'XBTUSD', 'side': 'Sell', 'clOrdID': 1538724150645130, 'ordType': 'Limit'}
[2018-10-05 15:22:32 bitmexWS_Trading INFO] Got position msg:
[2018-10-05 15:22:32 bitmexWS_Trading INFO] Got order msg
[2018-10-05 15:22:32 OMS INFO] Successfully Place Order:
{'timestamp': '2018-10-05T07:22:32.064Z', 'leavesQty': 0, 'cumQty': 4, 'simpleOrderQty': None, 'price': 6541.5, 'transactTime': '2018-10-05T07:22:32.064Z', 'clOrdID': '1538724150645130', 'side': 'Sell', 'settlCurrency': 'XBt', 'ordType': 'Limit', 'workingIndicator': False, 'symbol': 'XBTUSD', 'pegOffsetValue': None, 'orderQty': 4, 'orderID': '04730b85-d41d-7029-d769-fd4961e525c0', 'stopPx': None, 'execInst': '', 'triggered': '', 'clOrdLinkID': '', 'exDestination': 'XBME', 'simpleLeavesQty': 0, 'contingencyType': '', 'multiLegRe

Event: <__main__.MarketEvent object at 0x107a21ba8>


[2018-10-05 15:22:34 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:22:37 bitmexREST INFO] Canceling All Orders... symbol=XBTUSD, side=None


...

Event: <__main__.MarketEvent object at 0x107a49898>


[2018-10-05 15:23:27 bitmexWS_Trading DEBUG] >>> send ping
[2018-10-05 15:23:29 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:23:32 bitmexWS_Trading INFO] Got position msg:
[2018-10-05 15:23:32 bitmexWS_Trading DEBUG] >>> send ping
[2018-10-05 15:23:34 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:23:37 bitmexREST INFO] Canceling All Orders... symbol=XBTUSD, side=None


▲▲▲▲▲ target_posion: {'XBTUSD': 8} ▲▲▲▲▲


[2018-10-05 15:23:37 bitmexWS_Trading DEBUG] >>> send ping
[2018-10-05 15:23:39 bitmexREST INFO] Placing Order... post_dict is {'text': '[API][2018-10-05 15:23:39.191370] ', 'orderQty': 16, 'price': 6546.5, 'symbol': 'XBTUSD', 'side': 'Buy', 'clOrdID': 1538724219191366, 'ordType': 'Limit'}
[2018-10-05 15:23:39 bitmexWS_Market DEBUG] >>> send ping


KeyboardInterrupt: 

In [6]:
#bm_ws_market.exit()
#oms.bm_ws_trading.exit()

oms.exit()

[2018-10-05 15:23:43 bitmexWS_Trading INFO] Exiting ...
[2018-10-05 15:23:44 bitmexWS_Market DEBUG] >>> send ping
[2018-10-05 15:23:47 bitmexWS_Trading INFO] Got position msg:
[2018-10-05 15:23:47 bitmexWS_Trading INFO] ping thread end.
[2018-10-05 15:23:48 bitmexWS_Trading DEBUG] Calling ws.__on_close()
[2018-10-05 15:23:48 bitmexWS_Trading INFO] Exit bitmexWS (intended)
[2018-10-05 15:23:48 bitmexWS_Market INFO] Exiting ...
[2018-10-05 15:23:49 bitmexWS_Market INFO] ping thread end.
[2018-10-05 15:23:49 bitmexWS_Market DEBUG] Calling ws.__on_close()
[2018-10-05 15:23:49 bitmexWS_Market INFO] Exit bitmexWS (intended)


In [7]:
print(oms.bm_ws_trading.actual_position)
print(oms.bm_ws_trading.unfilled_qty)
print(oms.bm_ws_market.last_price)

{'XBTUSD': -8}
{'XBTUSD': {'Buy': 0, 'Sell': 0}}
6544
